# SISU Digital Humanities: Textual and Language Analysis on Social Media<br />
### Session 4: Word Embeddings
Created by Tom van Nuenen (tom.van_nuenen@kcl.ac.uk) <br />


# Word Embeddings

Today, we'll have a look at word embeddings using Gensim's `word2vec` and `doc2vec` methods. 

The goal of word vector embedding models is to learn dense, numerical vector representations for each term in a corpus vocabulary. If successful, the vectors for each term encode information about the meaning or concept the term represents, as well as the relationship between it and other terms in the vocabulary. Word vector models are  fully unsupervised: they learn all of these meanings and relationships without any advance knowledge.

After working through today's notebook, you'll be able to:

1. Use Gensim's word2vec method to create word vectors for a corpus;
2. Use these word vectors to reflect on implicit binaries and normativities in your data;
3. Visualize topic models using K-means clustering and t-SNE.

**Note: I stronly encourage you to use your own dataset to start figuring out semantic patterns that you are interested in!**

In [ ]:
# General
from pprint import pprint
from collections import Counter
import os
import re
import logging
import string
import pickle
import numpy as np
import pandas as pd
import smart_open
import multiprocessing 
from time import time  # To time our operations
from collections import defaultdict  # For word frequency

# Gensim
import gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from gensim.models.phrases import Phrases, Phraser

# NLTK
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer 
wordnet_lemmatizer = WordNetLemmatizer()

# Spacy
import spacy 

# Plotting
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource, value

# Clustering 
from sklearn.cluster import KMeans
from sklearn.neighbors import KDTree
from sklearn.manifold import TSNE

# Suppressing warnings
import warnings
warnings.simplefilter("ignore", DeprecationWarning)

## Preprocessing

We'll start by cleaning up our data a bit. Let's load it up.

In [ ]:
# load into df
df_com = pd.read_csv("data/TRP-comments.csv", lineterminator='\n')

In [ ]:
# Get rid of empty values and reset index
df_com = df_com[~df_com['body'].isin(['[removed]', '[deleted]' ])].dropna(subset=['body']).reset_index(drop=True)

Let's create a small function that cleans up our text by removing all escape-tabs and escape-newlines, as well as all non symbol characters (except for the dot). It also normalizes spaces to a single character and removes leading and trailing spaces.

In [ ]:
def clean_text(text):
    # Normalize tabs and remove newlines
    no_tabs = text.replace('\t', ' ').replace('\n', '');
    # Remove all characters except A-Z and a dot.
    alphas_only = re.sub("[^a-zA-Z\.]", " ", no_tabs);
    # Normalize spaces to 1
    multi_spaces = re.sub(" +", " ", alphas_only);
    # Strip trailing and leading spaces
    no_spaces = multi_spaces.strip();
    return no_spaces

We can now use the Pandas `.apply` method, allowing us to apply a function along an axis of the DataFrame. We'll use a lambda function: a small stand-in function that can take arguments, but only one expression. This is what a lambda looks like. Do you see how it works?

In [ ]:
df_com["body_clean"] = df_com["body"].apply(lambda x: clean_text(x))

We now have an additional column in our DataFrame with cleaned up text.

In [ ]:
df_com.head()

Let's turn it into a list.

In [ ]:
text_li = df_com['body_clean'].tolist()

Next, we'll create a function that uses NLTK's `sent_tokenize()` method. This tokenizer splits our texts into sentences, which in turn are split into tokens. We'll also remove stopwords.

In [ ]:
def sentence_tokenize(text):
    sentence_doc = sent_tokenize(text)
    sentences = [gensim.utils.simple_preprocess(str(doc), deacc=True) for doc in sentence_doc]  # deacc=True removes punctuations
    stop = set(stopwords.words('english') + ['’', '“', '”', 'nbsp', 'http'])
    no_stop = [[word for word in sentence if word not in stop] for sentence in sentences]
    return no_stop

In [ ]:
com_sent_li = [sentence_tokenize(text) for text in text_li]

Note that we now have a list (of comments) of lists (sentences) of lists (tokens). Let's index the first token of the first sentence of the first comment:

In [ ]:
com_sent_li[0][0][0]

We actually don't need the comment-level demarcation for the rest of our analysis. We can *flatten* our `com_sent_li` object to do so – this way, we create a list (of sentences) of lists (tokens).

In [ ]:
sent_li = []
for sentence in com_sent_li:
    for tokens in sentence:
        sent_li.append(tokens)

Writing the same in a list comprehension looks like this, by the way:

In [ ]:
sent_li = [tokens for sentence in com_sent_li for tokens in sentence]

Next, let's create a trigrams model using Gensim's `Phrases` and `Phraser` classes:

In [ ]:
bigram = Phrases(sent_li, min_count=5, threshold=80)
trigram = Phrases(bigram[sent_li], threshold=80)  
bigram_mod = Phraser(bigram)
trigram_mod = Phraser(trigram)

And let's run that model over our list of lists.

In [ ]:
trigrams = [trigram_mod[bigram_mod[sentence]] for sentence in sent_li]

## Word2Vec

Let's create our word embeddings model. Its input is a text corpus (split up in sentences) and its output is a set of "vectors" in N dimensions. It allows us to group the vectors of similar words together in vectorspace. We can then reduce the dimensionality to visualize the results in a way humans can understand (such as in a 2-dimensional space), or to perform linear algebra in order to find how words are related.

Word2vec is one example of a word embeddings model. It learns by taking words and their contexts (e.g. sentences) into account, and can then try to predict other words. Given enough data, usage and contexts, word2vec can make accurate guesses about a word’s meaning based on its appearances. Those guesses can be used to establish a word’s association with other words (e.g. “man” is to “boy” what “woman” is to “girl”), or cluster documents and classify them by topic.

### How many cores?
Word2Vec can work using independent threads doing simultaneous training. In general, you'll never want to use more workers than the number of CPU cores you have in your machine. So let's check out how many you have.



In [ ]:
cores = multiprocessing.cpu_count() # Count the number of cores in a computer
cores

We now instantiate and train our Word2Vec model, using the parameters below.

In [ ]:
num_features = 500        # Word vector dimensionality (how many features each word will be given)
min_word_count = 2        # Minimum word count to be taken into account
num_workers = cores       # Number of threads to run in parallel (equal to your amount of cores)
context = 10              # Context window size
downsampling = 0 #1e-2    # Downsample setting for frequent words
seed_n = 1                # Seed for the random number generator (to create reproducible results) 
sg_n = 1                  # Skip-gram = 1, CBOW = 0

model = Word2Vec(trigrams, workers=num_workers, \
            size=num_features, min_count = min_word_count, \
            window = context, sample = downsampling, seed=seed_n, sg=sg_n)

That was it! We can save this model in a Gensim object.

In [ ]:
model.save("word2vec.vec")

In [ ]:
model = Word2Vec.load("word2vec.vec")

How many terms are in our vocabulary?

In [ ]:
print('{:,} terms in the vocabulary.'.format(len(model.wv.vocab)))

### Getting related terms

With the information in our word embeddings model, we can try to find similarities between words that interest us (i.e. words that have a similar vector). Let's create a function that retrieves related terms to some input.

In [ ]:
def get_related_terms(token, topn=20):
    """
    look up the topn most similar terms to token and print them as a formatted list
    """

    for word, similarity in model.most_similar(positive=[token], topn=topn):
        print(word, round(similarity, 3))

In [ ]:
get_related_terms(u'woman')

### Word algebra

Word algebra, also known as analogy completion, means doing math with words (like the famous example "king - man + woman = queen". The core idea is that once words are represented as numerical vectors, you can do math with them. The mathematical procedure works as follows:

1. Provide a set of words or phrases you want to add or subtract.
2. Look up the vectors that represent those terms in the word vector model.
3. Add and subtract those vectors to produce a new, combined vector.
4. Look up the most similar vector(s) to this new, combined vector via cosine similarity.
5. Return the word(s) associated with the similar vector(s).

Let's try it out. We'll create a function that does this for us.

In [ ]:
def word_algebra(add=[], subtract=[], topn=1):
    """
    combine the vectors associated with the words provided
    in add= and subtract=, look up the topn most similar
    terms to the combined vector, and print the result(s)
    """
    answers = model.most_similar(positive=add, negative=subtract, topn=topn)
    
    for term, similarity in answers:
        print(term)

In [ ]:
word_algebra(add=['game','dating'])

In [ ]:
word_algebra(add=['game', 'dating'], subtract=['beta'])

## Word Vector Visualization with t-SNE

t-Distributed Stochastic Neighbor Embedding, or t-SNE, is a dimensionality reduction technique to assist with visualizing high-dimensional datasets. It attempts to map high-dimensional data onto a low two- or three-dimensional representation. It tries to keep the relative distances between points as closely as possible in both high-dimensional and low-dimensional space.

Scikit-learn provides a convenient implementation of the t-SNE algorithm with its `TSNE` class.

Our input for t-SNE will be the DataFrame of word vectors we created before. Let's first drop some stopwords, and
take only the 5,000 most frequent terms in the vocabulary for time's sake.

First, we need to create a DataFrame with the terms as the row labels, and the 100 dimensions of the word vector model as the columns.

In [ ]:
# build a list of the terms, integer indices, and term counts from the word2vec model vocabulary
ordered_vocab = [(term, voc.index, voc.count) for term, voc in model.wv.vocab.items()]

# sort by the term counts, so the most common terms appear first
ordered_vocab.sort(key = lambda x: x[2])  

# unzip the terms, integer indices, and counts into separate lists
ordered_terms, term_indices, term_counts = zip(*ordered_vocab)

# create a DataFrame with the vectors as data, and the terms as row labels
word_vectors = pd.DataFrame(model.wv.syn0norm[term_indices, :], index=ordered_terms)

word_vectors.head()

In [ ]:
tsne = TSNE()
tsne_vectors = tsne.fit_transform(word_vectors.values)

In [ ]:
tsne_vectors = pd.DataFrame(tsne_vectors,
                            index=pd.Index(word_vectors.index),
                            columns=['x_coord', 'y_coord'])

In [ ]:
tsne_vectors.head()

In [ ]:
tsne_vectors['word'] = tsne_vectors.index

In [ ]:
output_notebook()

In [ ]:
# add our DataFrame as a ColumnDataSource for Bokeh
plot_data = ColumnDataSource(tsne_vectors)

# create the plot and configure the
# title, dimensions, and tools
tsne_plot = figure(title='t-SNE Word Embeddings',
                   plot_width = 800,
                   plot_height = 800,
                   tools= ('pan, wheel_zoom, box_zoom, box_select, reset, reset'),
                   active_scroll='wheel_zoom')

# add a hover tool to display words on roll-over
tsne_plot.add_tools(HoverTool(tooltips = '@word'))

# draw the words as circles on the plot
tsne_plot.circle('x_coord', 'y_coord', source=plot_data,
                 color='blue', line_alpha=0.2, fill_alpha=0.1,
                 size=10, hover_line_color='black')

# configure visual elements of the plot
tsne_plot.title.text_font_size = value('14pt')
tsne_plot.xaxis.visible = False
tsne_plot.yaxis.visible = False
tsne_plot.grid.grid_line_color = None
tsne_plot.outline_line_color = None

# show the plot
show(tsne_plot);

## K-means clustering
Another thing we can do is cluster the words using K-Means clustering. 

K-Means clustering aims to partition N observations into K clusters in which each observation belongs to the cluster with the nearest mean (called the "cluster centre"), which serves as a prototype of the cluster.

Since our words are all represented as vectors, applying K-Means is easy to do since the clustering algorithm will simply look at differences between vectors (and centers).

In [ ]:
def clustering_on_wordvecs(word_vectors, num_clusters):
    # Initalize a k-means object and use it to extract centroids
    kmeans_clustering = KMeans(n_clusters = num_clusters, init='k-means++');
    idx = kmeans_clustering.fit_predict(word_vectors);
    
    return kmeans_clustering.cluster_centers_, idx;

In [ ]:
Z = model.wv.syn0

In [ ]:
centers, clusters = clustering_on_wordvecs(Z, 10);
centroid_map = dict(zip(model.wv.index2word, clusters));

Next, we get words in each cluster that are closest to the cluster center. To do this, we initialize a KDTree on the word vectors, and query it for the Top K words on each cluster center. Using the Index 2 word dictionary, we than correspond each word vector back to it’s original word representation and add them to a dataframe for easier printing.

In [ ]:
def get_top_words(index2word, k, centers, wordvecs):
    tree = KDTree(wordvecs);
    # Use closest points for each cluster center to query closest 20 points to it
    closest_points = [tree.query(np.reshape(x, (1, -1)), k=k) for x in centers];
    closest_words_idxs = [x[1] for x in closest_points];
    # Query Word Index  for each position in the above array, and added to a Dictionary
    closest_words = {};
    for i in range(0, len(closest_words_idxs)):
        closest_words['Cluster #' + str(i)] = [index2word[j] for j in closest_words_idxs[i][0]]
    # Create DataFrame from dictionary
    df = pd.DataFrame(closest_words);
    df.index = df.index+1
    return df

Let’s get the top words and print the first 20 in each cluster:

In [ ]:
top_words = get_top_words(model.wv.index2word, 5000, centers, Z);

In [ ]:
top_words[:10]

## **Doc2Vec**

You might be interested in the relationship between comments or posts, instead of single words. You can do this using tf-idf, as you saw in week 2. But word embeddings offer another unsupervised approach to find similarities.

Let's read our data again. We’ll need to associate a tag/number with each document of the training corpus. In our case, the tag is simply a zero-based line number.



In [ ]:
def read_text(inp):
    for i, line in enumerate(inp):
        tokens = gensim.utils.simple_preprocess(line)
        yield gensim.models.doc2vec.TaggedDocument(tokens, [i])

comments = list(read_text(df_com['body_clean']))

Instantiate a Doc2Vec model with a vector size with 50 dimensions and iterating over the training corpus 40 times. We set the minimum word count to 2 in order to discard words with very few occurrences.

In [ ]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=40)

Build the vocabulary:

In [ ]:
model.build_vocab(comments)

Train the model on the corpus:

In [ ]:
model.train(comments, total_examples=model.corpus_count, epochs=model.epochs)

To assess our new model, we’ll first infer new vectors for each document of the training corpus, compare the inferred vectors with the training corpus, and then returning the rank of the document based on self-similarity. Basically, we’re pretending as if the training corpus is some new unseen data and then seeing how they compare with the trained model. The expectation is that we’ve likely overfit our model (i.e., all of the ranks will be less than 2) and so we should be able to find similar documents very easily. Additionally, we’ll keep track of the second ranks for a comparison of less similar documents.

In [ ]:
ranks = []
second_ranks = []
for doc_id in range(len(comments)):
    inferred_vector = model.infer_vector(comments[doc_id].words)
    sims = model.docvecs.most_similar([inferred_vector], topn=len(model.docvecs))
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)

    second_ranks.append(sims[1])

Let's find some similar comments!

In [ ]:
print('Comment ({}): «{}»\n'.format(doc_id, ' '.join(comments[doc_id].words)))
print(u'SIMILAR/DISSIMILAR COMMENTS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('SECOND-MOST', 1), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(comments[sims[index][0]].words)))

And let's automate that a bit

In [ ]:
# Pick a random document from the corpus and infer a vector from the model
import random
doc_id = random.randint(0, len(comments) - 1)

# Compare and print the second-most-similar document
print('Train Document ({}): «{}»\n'.format(doc_id, ' '.join(comments[doc_id].words)))
sim_id = second_ranks[doc_id]
print('Similar Document {}: «{}»\n'.format(sim_id, ' '.join(comments[sim_id[0]].words)))
